# x402 facilitor for Optimism 

In this notebook, we test the x402 facilitator for Optimism network.

In [1]:
import requests
import json
from datetime import datetime

# Facilitator endpoint (local testing or deployed)
FACILITATOR_URL = "http://localhost:8080"  # Change to deployed URL when available

In [2]:
from web3 import Web3
from eth_account.messages import encode_typed_data
from eth_account import Account
import os
from dotenv import load_dotenv
import secrets

# Load environment variables
load_dotenv()
private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
pay_to_address = os.getenv('NFT_WALLET_PUBLIC_KEY')

# Create account from private key
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)
pay_to_address = Web3.to_checksum_address(pay_to_address)
print(f"Payer Address: {from_address}")
print(f"Recipient Address: {pay_to_address}")

# Network and token configuration
CHAIN_ID = 11155420  # Optimism Sepolia
USDC_ADDRESS = "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"  # USDC on Optimism Sepolia

# Create authorization data
valid_after = int(datetime.now().timestamp()) - 60  # Valid from 1 min ago
valid_before = int(datetime.now().timestamp()) + 3600  # Valid for 1 hour
nonce = "0x" + secrets.token_hex(32)  # Random nonce
amount = "10000"  # 0.01 USDC (6 decimals)

# EIP-712 domain for USDC
domain_data = {
    "name": "USD Coin",
    "version": "2",
    "chainId": CHAIN_ID,
    "verifyingContract": USDC_ADDRESS
}

# EIP-712 message (TransferWithAuthorization)
message_data = {
    "from": from_address,
    "to": pay_to_address,
    "value": int(amount),
    "validAfter": int(valid_after),
    "validBefore": int(valid_before),
    "nonce": bytes.fromhex(nonce[2:])  # Convert hex string to bytes
}

# EIP-712 types
types_data = {
    "EIP712Domain": [
        {"name": "name", "type": "string"},
        {"name": "version", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "verifyingContract", "type": "address"}
    ],
    "TransferWithAuthorization": [
        {"name": "from", "type": "address"},
        {"name": "to", "type": "address"},
        {"name": "value", "type": "uint256"},
        {"name": "validAfter", "type": "uint256"},
        {"name": "validBefore", "type": "uint256"},
        {"name": "nonce", "type": "bytes32"}
    ]
}

# Create the full typed data structure
typed_data = {
    "types": types_data,
    "primaryType": "TransferWithAuthorization",
    "domain": domain_data,
    "message": message_data
}

# Encode and sign
encoded_data = encode_typed_data(full_message=typed_data)
signed_message = account.sign_message(encoded_data)

# Add 0x prefix to signature (required by viem)
signature_hex = "0x" + signed_message.signature.hex() if not signed_message.signature.hex().startswith("0x") else signed_message.signature.hex()

print(f"\nSignature: {signature_hex}")
print(f"Nonce: {nonce}")
print(f"Valid After: {valid_after}")
print(f"Valid Before: {valid_before}")

Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

Signature: 0xaff07e1f79e60e67b35c1bfbed2bd618239576a63407186dedcef5838107b87038de996f0dd6dde3f4f1ed788574e9c90943e80ab76d49846cbc4c84e5937eca1c
Nonce: 0xcaffdec119e38ec5e7a62fbf269b4209913133c7414f4079f4fa61165cd403e4
Valid After: 1765998213
Valid Before: 1766001873


### Debug: Prüfe ob Nonce bereits verwendet wurde

Manchmal wurde eine Nonce bereits in einem früheren Test verwendet. Lass uns prüfen:

In [ ]:
# Check if the current nonce was already used on-chain
print(f"Prüfe Nonce: {nonce}")
print(f"Payer: {from_address}")

# USDC contract ABI for authorizationState
usdc_abi = [
    {
        "inputs": [
            {"name": "authorizer", "type": "address"},
            {"name": "nonce", "type": "bytes32"}
        ],
        "name": "authorizationState",
        "outputs": [{"name": "", "type": "bool"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Connect to Optimism Sepolia
w3 = Web3(Web3.HTTPProvider('https://sepolia.optimism.io'))
usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=usdc_abi)

# Check if nonce is used
nonce_used = usdc_contract.functions.authorizationState(from_address, nonce).call()

if nonce_used:
    print(f"\n❌ Diese Nonce wurde bereits verwendet!")
    print(f"Du musst die vorherige Zelle erneut ausführen, um eine neue Nonce zu generieren.")
else:
    print(f"\n✅ Diese Nonce ist noch verfügbar und kann verwendet werden.")

In [3]:
# Now create a valid x402 payment payload with our real signature
real_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": "eip155:11155420",  # Optimism Sepolia testnet
        "amount": amount,
        "asset": USDC_ADDRESS,
        "payTo": pay_to_address,
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": "USDC",
            "version": "2"
        }
    },
    "payload": {
        "signature": signature_hex,
        "authorization": {
            "from": from_address,
            "to": pay_to_address,
            "value": amount,
            "validAfter": str(valid_after),
            "validBefore": str(valid_before),
            "nonce": nonce
        }
    }
}

# Create payment requirements
real_payment_requirements = {
    "scheme": "exact",
    "network": "eip155:11155420",
    "amount": amount,
    "asset": USDC_ADDRESS,
    "payTo": pay_to_address,
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": "USDC",
        "version": "2"
    }
}

# Create verify request
real_verify_request = {
    "paymentPayload": real_payment_payload,
    "paymentRequirements": real_payment_requirements
}

# Send to facilitator
response = requests.post(
    f"{FACILITATOR_URL}/verify",
    headers={"Content-Type": "application/json"},
    json=real_verify_request
)

print(f"Status Code: {response.status_code}")
result = response.json()
print(json.dumps(result, indent=2))

if result.get("isValid"):
    print(f"\n✅ Payment ist GÜLTIG!")
    print(f"Payer: {result.get('payer')}")
else:
    print(f"\n❌ Payment ist UNGÜLTIG")
    print(f"Grund: {result.get('invalidReason')}")

Status Code: 200
{
  "isValid": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

✅ Payment ist GÜLTIG!
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101


## Settlement via Facilitator

Nach erfolgreicher Verifikation kann der Facilitator die Zahlung on-chain ausführen. Der `/settle` Endpoint:
1. Führt nochmals alle Verifikationen durch
2. Ruft `transferWithAuthorization` auf dem USDC-Contract auf
3. Gibt die Transaction Hash zurück

**Voraussetzungen für Settlement:**
- Der Payer braucht USDC auf Optimism Sepolia
- Der Facilitator braucht einen Private Key (`FACILITATOR_WALLET_PRIVATE_KEY` in `.env`) und etwas ETH für Gas
- Für diesen Test kannst du deinen eigenen Private Key verwenden (im echten Betrieb sollte der Facilitator einen separaten Wallet haben)

### Prüfe Facilitator-Wallet Balance

Bevor wir Settlement versuchen, prüfen wir ob der Facilitator genug ETH für Gas hat:

In [4]:
# Check facilitator wallet balance using public key from .env
facilitator_address = os.getenv('FACILITATOR_WALLET_PUBLIC_KEY')

if facilitator_address:
    facilitator_address = Web3.to_checksum_address(facilitator_address)
    
    # Connect to Optimism Sepolia to check balance
    w3 = Web3(Web3.HTTPProvider('https://sepolia.optimism.io'))
    
    if w3.is_connected():
        balance_wei = w3.eth.get_balance(facilitator_address)
        balance_eth = w3.from_wei(balance_wei, 'ether')
        
        print(f"Facilitator Wallet: {facilitator_address}")
        print(f"ETH Balance: {balance_eth} ETH")
        
        # Estimate gas cost (rough estimate: ~100k gas)
        estimated_gas = 100000
        gas_price = w3.eth.gas_price
        estimated_cost_wei = estimated_gas * gas_price
        estimated_cost_eth = w3.from_wei(estimated_cost_wei, 'ether')
        
        print(f"Geschätzte Gas-Kosten: ~{estimated_cost_eth:.6f} ETH")
        
        if balance_wei == 0:
            print(f"\n❌ FEHLER: Facilitator Wallet hat kein ETH!")
            print(f"Sende ETH an: {facilitator_address}")
            print(f"Bridge: https://app.optimism.io/bridge")
        elif balance_wei < estimated_cost_wei:
            print(f"\n⚠️  WARNUNG: Balance könnte zu niedrig für Gas sein")
            print(f"Empfohlen: Mindestens {estimated_cost_eth:.6f} ETH")
        else:
            print(f"\n✅ Ausreichend ETH für Settlement vorhanden")
    else:
        print("❌ Konnte nicht mit Optimism Sepolia verbinden")
else:
    print("❌ FACILITATOR_WALLET_PUBLIC_KEY nicht in .env konfiguriert")

Facilitator Wallet: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206
ETH Balance: 0.05 ETH
Geschätzte Gas-Kosten: ~0.000000 ETH

✅ Ausreichend ETH für Settlement vorhanden


In [5]:
# Try to settle the payment (execute on-chain)
# Note: This will fail if the payer doesn't have USDC or if the facilitator wallet has no ETH for gas

settle_response = requests.post(
    f"{FACILITATOR_URL}/settle",
    headers={"Content-Type": "application/json"},
    json=real_verify_request  # Same request structure as verify
)

print(f"Status Code: {settle_response.status_code}")
settle_result = settle_response.json()
print(json.dumps(settle_result, indent=2))

if settle_result.get("success"):
    print(f"\n✅ Settlement ERFOLGREICH!")
    print(f"Transaction Hash: {settle_result.get('transaction')}")
    print(f"Network: {settle_result.get('network')}")
    print(f"\nSieh dir die Transaktion an auf:")
    print(f"https://sepolia-optimism.etherscan.io/tx/{settle_result.get('transaction')}")
else:
    print(f"\n❌ Settlement FEHLGESCHLAGEN")
    print(f"Grund: {settle_result.get('errorReason')}")
    if settle_result.get('errorReason') == 'insufficient_funds':
        print(f"\nDer Payer ({settle_result.get('payer')}) hat nicht genug USDC.")
        print(f"Hole dir Test-USDC auf Optimism Sepolia:")
        print(f"1. Bridge von Sepolia ETH → Optimism Sepolia: https://app.optimism.io/bridge")
        print(f"2. Swap ETH → USDC auf Uniswap: https://app.uniswap.org/")

Status Code: 200
{
  "success": false,
  "errorReason": "authorization_already_used",
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "",
  "network": "eip155:11155420"
}

❌ Settlement FEHLGESCHLAGEN
Grund: authorization_already_used
